In [1]:
import exchange_calendars as ecals
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from collections import Counter

## 1. 뉴스 데이터 불러오기

In [3]:
newsdata = pd.read_csv('../data/newsfile_nouns.csv')
newsdata.head(3)

,Unnamed: 0,date,title,nouns
0,19650,20151019,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
1,3420,20151019,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
2,1798,20151019,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상


In [5]:
newsdata.drop(columns=['Unnamed: 0'], inplace=True)
newsdata.head(3)

,date,title,nouns
0,20151019,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
1,20151019,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
2,20151019,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상


In [11]:
# 날짜 컬럼을 리스트 형식으로 변환
newsdate = newsdata['date'].to_list()
newsdate[:5]

[20151019, 20151019, 20151019, 20151019, 20151019]

## 2. 한국 거래소 개장일 확인 코드

In [2]:
# 한국코드
XKRX = ecals.get_calendar("XKRX")       
# 2021-01-01은 개장일인지 확인
print("2021-01-01 개장 확인: ", XKRX.is_session('2021-01-01'))    
# 오늘(22-03-19)은 개장일인지 확인
print("오늘날짜 개장일 확인: ", XKRX.is_session(datetime.date.today().strftime('%Y-%m-%d')))  
# 다음 개장일은 언제인지 확인
print("다음 개장일 날짜 확인: ", XKRX.next_open(pd.Timestamp.today().strftime('%Y-%m-%d')))

2021-01-01 개장 확인:  False
오늘날짜 개장일 확인:  False
다음 개장일 날짜 확인:  2022-03-21 00:00:00+00:00


In [13]:
# 뉴스데이터 날짜 다음날에 해당하는 개장일 날짜 확인
next_open_list = []

for day in range(0, len(newsdate)):
    next_open_date = XKRX.next_open(day)
    next_open_list.append(next_open_date)

MinuteOutOfBounds: Parameter `dt` receieved as '1970-01-01 00:00:00+00:00' although cannot be earlier than the first trading minute of calendar 'XKRX' ('2002-03-20 00:00:00+00:00').